## import package

In [9]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

## Two Layer Net
- 전체 : 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 테이터에 적응하도록 조정하는 과정을 '학습'이라고 합니다. 신경망의 학습은 다음과 같이 4단계로 구성
- 1단계 - 미니배치 : 훈련 데이터 중 일부를 무작위로 가져옴. 이렇게 선별한 데이터를 미니배치라고 하며 그 미니배치의 손실함수값을 줄이는 것이 목표
- 2단계 - 기울기 산출 : 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 뱡향 제기
- 3단계 - 매개변수 갱신 : 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
- 4단계 - 반복 : 1~3단계 반복

In [17]:
class TwoLayerNet():
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # initialize weights
        # params: 신경망의 매개변수를 보관하는 딕셔너리 변수
        self.params = {}
        self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        # predict
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        # grads: 기울기를 보관하는 딕셔너리 변수
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

## TwoLayerNet working check

In [18]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [19]:
# Check for predict
x = np.random.rand(100, 784)
y = net.predict(x)
print(y.shape)

(100, 10)


In [20]:
# Check for Gradient
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x,t)

grads['W1'].shape

(784, 100)

# Test Learning with Numerical Gradient

## Load package

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from TwoLayerNet_module import TwoLayerNet

## MNIST Loader

In [28]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

## Hyperparameter

In [31]:
train_loss_list = []
iter_list = []
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

## Define TwoLayerNet

In [32]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

## Mini-batch Learning
- 20일 걸림

In [ ]:
for i in range(iters_num):
    # for Debugging
    print(i)
    
    # Mini-batch setup
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # Compute Gradient
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # SGD
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
        
    # Record the history
    loss = network.loss(x_batch, t_batch)
    iter_list.append(i)
    train_loss_list.append(loss)
    
plt.plot(iter_list, train_loss_list)